In [221]:
import os
import pandas as pd
from sklearn import svm
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score
from collections import defaultdict
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
import random

# Predicting Games with Separate Models

In [333]:
# Note for predictions, but never used in code

tournament_result_key = {"0" : "Didn't qualify", "1": "Round of 64", "2": "Round of 32", "4": "Sweet 16", "5": "Elite 8", "7": "Final 4",
                        "8": "Last 2", "9": "Champion"}
tournament_result_after_2010 = {"0" : "Didn't qualify", "1": "First Four", "2": "Round of 64", "3": "Round of 32", "4": "Sweet 16", "5": "Elite 8", "7": "Final 4",
                        "8": "Last 2", "9": "Champion"}

In [392]:
# Read in data
df = pd.read_csv("data/all years normalized.csv")

df = df.rename({'Tourament-Result': 'Tournament-Result'}, axis = 1)
df

/var/folders/q9/42jptm7d151_h70_cp05s8dm0000gn/T/ipykernel_80444/3167189561.py:2: DtypeWarning: Columns (55,56,57,58,65,75,82,83,89,93) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/all years normalized.csv")


,Season,School,Conference,AP-Pre,AP-High,AP-Final,Tournament-Result,Tournament-Seed,G,W,...,3PAr Normalized,TS% Normalized,TRB% Normalized,AST% Normalized,STL% Normalized,BLK% Normalized,eFG% Normalized,TOV% Normalized,ORB% Normalized,FT/FGA Normalized
0,1992-93,air-force,WAC,30.0,30.0,30.0,0.0,20.0,28,9,...,0.446809,0.306667,0.503311,0.205047,#DIV/0!,0.841155,0.219355,0.403226,#DIV/0!,0.418502
1,1992-93,akron,MAC,30.0,30.0,30.0,0.0,20.0,26,8,...,0.440729,0.420000,0.582781,0.520505,#DIV/0!,0.693141,0.412903,0.483871,#DIV/0!,0.321586
2,1992-93,alabama,SEC,30.0,30.0,30.0,0.0,20.0,29,16,...,0.507599,0.580000,0.622517,0.167192,#DIV/0!,0.815884,0.574194,0.564516,#DIV/0!,0.453744
3,1992-93,alabama-state,SWAC,30.0,30.0,30.0,0.0,20.0,27,14,...,0.215805,0.686667,0.596026,0.271293,#DIV/0!,0.595668,0.606452,0.669355,#DIV/0!,0.603524
4,1992-93,alcorn-state,SWAC,30.0,30.0,30.0,0.0,20.0,27,7,...,0.094225,0.420000,0.370861,0.466877,#DIV/0!,0.754513,0.406452,0.588710,#DIV/0!,0.088106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8556,2017-18,wright-state,Horizon,30.0,30.0,30.0,1.0,14.0,35,25,...,0.332258,0.382022,0.613333,0.547112,0.471264,0.176871,0.363128,0.483146,0.505,0.600000
8557,2017-18,wyoming,MWC,30.0,30.0,30.0,0.0,20.0,33,20,...,0.661290,0.578652,0.300000,0.747720,0.356322,0.401361,0.541899,0.460674,0.02,0.862500
8558,2017-18,xavier,Big East,17.0,3.0,3.0,2.0,1.0,35,29,...,0.364516,0.786517,0.833333,0.686930,0.310345,0.306122,0.743017,0.359551,0.57,1.000000
8559,2017-18,yale,Ivy,30.0,30.0,30.0,0.0,20.0,31,16,...,0.570968,0.567416,0.480000,0.799392,0.45977,0.292517,0.597765,0.505618,0.36,0.381250


In [393]:
# Obtaining list of schools in order to normalize vectors
school_list = []
conference_list = []

for i in range(2,17):
    if i < 9:
        practice_season = "200{}-0{}".format(i, i + 1)
    elif i == 9:
        practice_season = "200{}-{}".format(i, i + 1)
    else:
        practice_season = "20{}-{}".format(i, i + 1)
    # Get a filter year
    practice_year = df.loc[df["Season"].str.contains(practice_season)]
    practice_year = practice_year.fillna(0)
                           
    # Get total list of schools to normalize data
    practice_schools = practice_year["School"].to_numpy()
    
    practice_conf = practice_year["Conference"].to_numpy()
    
    for school in practice_schools:
        if school not in school_list:
            school_list.append(school)
            
    for conf in practice_conf:
    
        if conf not in conference_list:
            if conf == 0:
                conf = "Ind"
            conference_list.append(conf)



school_list = list(set(school_list))
school_list.sort()
conference_list = list(set(conference_list))
conference_list.sort()

In [394]:
print(school_list)

['abilene-christian', 'air-force', 'akron', 'alabama', 'alabama-a&m', 'alabama-state', 'albany-(ny)', 'alcorn-state', 'american', 'appalachian-state', 'arizona', 'arizona-state', 'arkansas', 'arkansas-pine-bluff', 'arkansas-state', 'army', 'auburn', 'austin-peay', 'ball-state', 'baylor', 'belmont', 'bethune-cookman', 'binghamton', 'birmingham-southern', 'boise-state', 'boston-college', 'boston-university', 'bowling-green-state', 'bradley', 'brigham-young', 'brown', 'bryant', 'bucknell', 'buffalo', 'butler', 'cal-poly', 'cal-state-bakersfield', 'cal-state-fullerton', 'cal-state-northridge', 'california', 'campbell', 'canisius', 'centenary-(la)', 'central-arkansas', 'central-connecticut-state', 'central-florida', 'central-michigan', 'charleston-southern', 'charlotte', 'chattanooga', 'chicago-state', 'cincinnati', 'clemson', 'cleveland-state', 'coastal-carolina', 'colgate', 'college-of-charleston', 'colorado', 'colorado-state', 'columbia', 'connecticut', 'coppin-state', 'cornell', 'creigh

In [395]:
# Set up school vectors for predicting the winners including names
# And to check winners 
school_indices = dict((c, float(i)) for i, c in enumerate(school_list))
indices_school = dict((float(i), c) for i, c in enumerate(school_list))

In [396]:
# Set up conference vectors for predicting the winners with conferences
conference_indices = dict((c, float(i)) for i, c in enumerate(conference_list))
indices_conference = dict((float(i), c) for i, c in enumerate(conference_list))
print(conference_indices)

{'A-10': 0.0, 'A-Sun': 1.0, 'AAC': 2.0, 'ACC': 3.0, 'AEC': 4.0, 'Big 12': 5.0, 'Big East': 6.0, 'Big Sky': 7.0, 'Big South': 8.0, 'Big Ten': 9.0, 'Big West': 10.0, 'CAA': 11.0, 'CUSA': 12.0, 'GWC': 13.0, 'Horizon': 14.0, 'Ind': 15.0, 'Ivy': 16.0, 'MAAC': 17.0, 'MAC': 18.0, 'MEAC': 19.0, 'MVC': 20.0, 'MWC': 21.0, 'Mid-Cont': 22.0, 'NEC': 23.0, 'OVC': 24.0, 'Pac-10': 25.0, 'Pac-12': 26.0, 'Patriot': 27.0, 'SEC': 28.0, 'SWAC': 29.0, 'Southern': 30.0, 'Southland': 31.0, 'Summit': 32.0, 'Sun Belt': 33.0, 'WAC': 34.0, 'WCC': 35.0}


In [398]:
# Setting up dictionary to connect teams to their data based on names
connecting_data = defaultdict()
tournament_teams = df


for i in range(2,17):
    if i < 9:
        practice_season = "200{}-0{}".format(i, i + 1)
    elif i == 9:
        practice_season = "200{}-{}".format(i, i + 1)
    else:
        practice_season = "20{}-{}".format(i, i + 1)
        
    year_data = defaultdict()
    year_teams = defaultdict()
    year = str(i+1)
    # Get a filter year
    practice_year = tournament_teams.loc[tournament_teams["Season"].str.contains(practice_season)]
    practice_year = practice_year.fillna(0)
    practice_year = practice_year.reset_index()

    practice_year = practice_year.drop(["Season"], axis =1)
    for ind in practice_year.index:
        
        team = practice_year["School"][ind]

        practice_year["School"][ind] = school_indices[team]
        
        if practice_year["Conference"][ind] == 0:

            practice_year["Conference"][ind] = conference_indices["Southland"]
        else:
            practice_year["Conference"][ind] = conference_indices[practice_year["Conference"][ind]]
        #print(ind, team)
        stats = practice_year.iloc[ind, :]
        # stats = stats.drop("School")
        
        
        drop_columns = [col for col in stats.index if "Normalized" not in col]
        drop_columns.remove("Conference")
        drop_columns.remove("School")

        drop_columns.append('Pace Normalized')
        drop_columns.append('ORtg Normalized')
        drop_columns.append('ORB% Normalized')
        drop_columns.append('STL% Normalized')
        drop_columns.append('3P Per Game Normalized')
        drop_columns.append('FG Per Game Normalized')
        drop_columns.append('FGA Per Game Normalized')
#         drop_columns.append('MP Normalized')
        drop_columns.append('BLK% Normalized')
        drop_columns.append('TRB% Normalized')
        drop_columns.append('FT/FGA Normalized')
        drop_columns.append('FTA Per Game Normalized')
        drop_columns.append('FT Per Game Normalized')
        drop_columns.append('W-Conf Normalized')
        drop_columns.append('L-Conf Normalized')
        drop_columns.append('W-Away Normalized')
        drop_columns.append('L-Away Normalized')
        drop_columns.append('W-Home Normalized')
        drop_columns.append('L-Home Normalized')
        drop_columns.append('TOV% Normalized')
        drop_columns.append('FTr Normalized')
        drop_columns.append('BLK Per Game Normalized')
        drop_columns.append('PF Per Game Normalized')
        drop_columns.append('3PA Per Game Normalized')
        drop_columns.append('W-L% Normalized')
        drop_columns.append('TRB Per Game Normalized')
        drop_columns.append('eFG% Normalized')
        drop_columns.append('AST% Normalized')
        drop_columns.append('TS% Normalized')
        drop_columns.append('3PAr Normalized')
        drop_columns.append('FT% Normalized')
        drop_columns.append('STL Per Game Normalized')
        drop_columns.append('TOV Per Game Normalized')
        drop_columns.append('ORB Per Game Normalized')
        drop_columns.append('AST Per Game Normalized')
        drop_columns.append('School')
        drop_columns.append('Conference')
        drop_columns.append('W Normalized')
        drop_columns.append('L Normalized')
        drop_columns.append('SRS Normalized')
#         drop_columns.append('SOS Normalized')
        drop_columns.append('Pts-Tm. Per Game Normalized')
        drop_columns.append('Pts-Opp. Per Game Normalized')
        drop_columns.append('FG% Normalized')
        drop_columns.append('3P% Normalized')
        
        for stat in stats.index:
            if isinstance(stats[stat], str):
                try:
                    stats[stat] = float(stats[stat])
                except:
                    print(stat)
                    drop_columns.append(stat)
            
        
        stats = stats.drop(index=drop_columns)
        print(stats.index)
        year_data[team] = stats
        
    connecting_data[practice_season] = year_data


/var/folders/q9/42jptm7d151_h70_cp05s8dm0000gn/T/ipykernel_80444/2517089757.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  practice_year["School"][ind] = school_indices[team]
/var/folders/q9/42jptm7d151_h70_cp05s8dm0000gn/T/ipykernel_80444/2517089757.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  practice_year["Conference"][ind] = conference_indices[practice_year["Conference"][ind]]
/var/folders/q9/42jptm7d151_h70_cp05s8dm0000gn/T/ipykernel_80444/2517089757.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.

Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object

/var/folders/q9/42jptm7d151_h70_cp05s8dm0000gn/T/ipykernel_80444/2517089757.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  practice_year["School"][ind] = school_indices[team]
/var/folders/q9/42jptm7d151_h70_cp05s8dm0000gn/T/ipykernel_80444/2517089757.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  practice_year["Conference"][ind] = conference_indices[practice_year["Conference"][ind]]
/var/folders/q9/42jptm7d151_h70_cp05s8dm0000gn/T/ipykernel_80444/2517089757.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.

Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object

/var/folders/q9/42jptm7d151_h70_cp05s8dm0000gn/T/ipykernel_80444/2517089757.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  practice_year["School"][ind] = school_indices[team]
/var/folders/q9/42jptm7d151_h70_cp05s8dm0000gn/T/ipykernel_80444/2517089757.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  practice_year["Conference"][ind] = conference_indices[practice_year["Conference"][ind]]
/var/folders/q9/42jptm7d151_h70_cp05s8dm0000gn/T/ipykernel_80444/2517089757.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.

...

/var/folders/q9/42jptm7d151_h70_cp05s8dm0000gn/T/ipykernel_80444/2517089757.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  practice_year["School"][ind] = school_indices[team]
/var/folders/q9/42jptm7d151_h70_cp05s8dm0000gn/T/ipykernel_80444/2517089757.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  practice_year["Conference"][ind] = conference_indices[practice_year["Conference"][ind]]
/var/folders/q9/42jptm7d151_h70_cp05s8dm0000gn/T/ipykernel_80444/2517089757.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.

Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object')
Index(['SOS Normalized', 'MP Normalized'], dtype='object

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [399]:
# Round to connect a prediction on based on its index
predictions_to_round = {"0": 0, "1": 1, "2": 2, "3":3, "4": 4, "5" : 5, "6": 7, "7": 8, "8": 9}
round_to_predict = {"0":0, "1":1, "2":2, "3":3, "4":4, "5":5, "7":6, "8":7, "9": 8}

In [400]:
# Preprocessing to try and make it so it is game by game instead
# Note data already includes play in games (so no first four)
rounds = pd.read_csv("data/big_dance.csv")

rounds = rounds.dropna(how='all') #remove empty rows

# Obtain only years with practice data

rounds["Start_Year"] = rounds["Season"].str.split('-').str[0].astype(float)
rounds["End_Year"] = rounds["Season"].str.split('-').str[1].astype(float)


In [430]:
def get_full_bracket(which_round):
    """ Returns the full bracket for a tournament with each included
    """
    round_results = []
    for ind in which_round.index:
        team1 = which_round["School"][ind]
        score1 = which_round["Score"][ind]
        team2 = which_round["School.1"][ind]
        score2 = which_round["Score.1"][ind]
        
        region_name = which_round["Region Name"][ind]
        path = which_round["Seed"][ind]
        
        r1 = random.randint(0, 100)

        if r1 %2 == 0:
            
            if score1 > score2:
                winner = (1,0)
            else:
                winner = (0,1)
            round_results.append([(team1, team2), winner, (path, 17-path), region_name])
        else:

            if score1 > score2:
                winner = (0,1)
            else:
                winner = (1,0)
            round_results.append([(team2, team1), winner, (17-path, path), region_name])

    return round_results

In [427]:
# Test year data
rounds_2017 = rounds.loc[rounds["Season"] == "2017-2018"]
rounds_2017


,Season,Round,Region Number,Region Name,Seed,Score,School,School.1,Score.1,Seed.1,Unnamed: 10,Unnamed: 11,Unnamed: 12,Start_Year,End_Year
1575,2017-2018,1.0,1.0,South,1.0,54.0,virginia,maryland-baltimore-county,74.0,16.0,NaN,NaN,NaN,2017.0,2018.0
1576,2017-2018,1.0,1.0,South,2.0,68.0,cincinnati,georgia-state,53.0,15.0,NaN,NaN,NaN,2017.0,2018.0
1577,2017-2018,1.0,1.0,South,3.0,73.0,tennessee,wright-state,47.0,14.0,NaN,NaN,NaN,2017.0,2018.0
1578,2017-2018,1.0,1.0,South,4.0,68.0,arizona,buffalo,89.0,13.0,NaN,NaN,NaN,2017.0,2018.0
1579,2017-2018,1.0,1.0,South,5.0,78.0,kentucky,davidson,73.0,12.0,NaN,NaN,NaN,2017.0,2018.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1633,2017-2018,4.0,3.0,East,1.0,71.0,villanova,texas-tech,59.0,3.0,NaN,NaN,NaN,2017.0,2018.0
1634,2017-2018,4.0,4.0,Midwest,1.0,85.0,kansas,duke,81.0,2.0,NaN,NaN,NaN,2017.0,2018.0
1635,2017-2018,5.0,1.0,Final Four,11.0,57.0,loyola-(il),michigan,69.0,3.0,NaN,NaN,NaN,2017.0,2018.0
1636,2017-2018,5.0,2.0,Final Four,1.0,95.0,villanova,kansas,79.0,1.0,NaN,NaN,NaN,2017.0,2018.0


In [428]:
# Get bracket data for our test year
full_bracket = []
for i in range(1,7):
    round_i = rounds_2017.loc[rounds_2017["Round"] == i]
    full_bracket.append(get_full_bracket(round_i))
print(full_bracket)

[[[('virginia', 'maryland-baltimore-county'), (0, 1), (1.0, 16.0), 'South'], [('georgia-state', 'cincinnati'), (0, 1), (15.0, 2.0), 'South'], [('wright-state', 'tennessee'), (0, 1), (14.0, 3.0), 'South'], [('buffalo', 'arizona'), (1, 0), (13.0, 4.0), 'South'], [('kentucky', 'davidson'), (1, 0), (5.0, 12.0), 'South'], [('loyola-(il)', 'miami-(fl)'), (1, 0), (11.0, 6.0), 'South'], [('texas', 'nevada'), (0, 1), (10.0, 7.0), 'South'], [('creighton', 'kansas-state'), (0, 1), (8.0, 9.0), 'South'], [('texas-southern', 'xavier'), (0, 1), (16.0, 1.0), 'West'], [('north-carolina', 'lipscomb'), (1, 0), (2.0, 15.0), 'West'], [('montana', 'michigan'), (0, 1), (14.0, 3.0), 'West'], [('unc-greensboro', 'gonzaga'), (0, 1), (13.0, 4.0), 'West'], [('south-dakota-state', 'ohio-state'), (0, 1), (12.0, 5.0), 'West'], [('houston', 'san-diego-state'), (1, 0), (6.0, 11.0), 'West'], [('texas-a&m', 'providence'), (1, 0), (7.0, 10.0), 'West'], [('florida-state', 'missouri'), (1, 0), (9.0, 8.0), 'West'], [('radfo

In [429]:
# Setting up practice data for each of our rounds 
practice_data_1 = []
practice_data_2 = []
practice_data_3 = []
practice_data_4 = []
practice_data_5 = []
practice_data_6 = []
practice_target_1 = []
practice_target_2 = []
practice_target_3 = []
practice_target_4 = []
practice_target_5 = []
practice_target_6 = []
for i in range(2,16):
    if i < 9:
        practice_season = "200{}-200{}".format(i, i + 1)
        year = "200{}-0{}".format(i, i + 1)
    elif i == 9:
        practice_season = "200{}-20{}".format(i, i + 1)
        year = "200{}-{}".format(i, i + 1)
    else:
        practice_season = "20{}-20{}".format(i, i + 1)
        year = "20{}-{}".format(i, i + 1)

    year_data = connecting_data[year]
    full_bracket = []

    rounds_year = rounds.loc[rounds["Season"] == practice_season]

    for i in range(1,7):
        round_i = rounds_year.loc[rounds_year["Round"] == i]

        full_bracket.append(get_full_bracket(round_i))

    for i, team_rounds in enumerate(full_bracket):
        
        for game in team_rounds:
            teams = game[0]
            winners = game[1]
            team1 = teams[0]
            team2 = teams[1]

            # Randomizes the order of where the a certain team is put for practice data
            r1 = random.randint(0, 100)
            order = 0
            if r1 %2 == 0:
                team1 = teams[0]
                team2 = teams[1]
                order = 0
            else:
                team2 = teams[0]
                team1 = teams[1]
                order = 1
                
            data1, data2 = year_data[team1], year_data[team2]
            data = np.array(data1 + data2)

            if i == 1:
                practice_data_1.append(data)
                practice_target_1.append(winners[order])
            elif i == 2:
                practice_data_2.append(data)
                practice_target_2.append(winners[order])
            elif i == 3:
                practice_data_3.append(data)
                practice_target_3.append(winners[order])
            elif i == 4:
                practice_data_4.append(data)
                practice_target_4.append(winners[order])
            elif i == 5:
                practice_data_5.append(data)
                practice_target_5.append(winners[order])
            elif i == 6:
                practice_data_6.append(data)
                practice_target_6.append(winners[order])

X_1 = np.array(practice_data_1)
Y_1 = np.array(practice_target_1)
X_2 = np.array(practice_data_2)
Y_2 = np.array(practice_target_2)
X_3 = np.array(practice_data_3)
Y_3 = np.array(practice_target_3)
X_4 = np.array(practice_data_4)
Y_4 = np.array(practice_target_4)
X_5 = np.array(practice_data_5)
Y_5 = np.array(practice_target_5)
 

In [405]:
# Train first round model
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_1, Y_1, test_size=0.2, random_state=42)
model1 = SGDClassifier(random_state = 42)
model1.fit(X_train_1, y_train_1)
predictions1 = model1.predict(X_test_1)
accuracy = accuracy_score(y_test_1, predictions1)
report = classification_report(y_test_1, predictions1)
print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)
print(predictions1)


Accuracy: 0.5111111111111111
Classification Report:
              precision    recall  f1-score   support

           0       0.51      1.00      0.68        23
           1       0.00      0.00      0.00        22

    accuracy                           0.51        45
   macro avg       0.26      0.50      0.34        45
weighted avg       0.26      0.51      0.35        45

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0]


/Users/eddieloyd/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/eddieloyd/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/eddieloyd/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [406]:
# Train 2nd round model
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2, Y_2, test_size=0.2, random_state=42)
model2 = SGDClassifier(random_state = 42)
model2.fit(X_train_2, y_train_2)
predictions2 = model2.predict(X_test_2)
accuracy = accuracy_score(y_test_2, predictions2)
report = classification_report(y_test_2, predictions2)
print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)
print(predictions2)

Accuracy: 0.5217391304347826
Classification Report:
              precision    recall  f1-score   support

           0       0.48      1.00      0.65        10
           1       1.00      0.15      0.27        13

    accuracy                           0.52        23
   macro avg       0.74      0.58      0.46        23
weighted avg       0.77      0.52      0.43        23

[0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [407]:
# Train 3rd round model
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X_3, Y_3, test_size=0.2, random_state=42)
model3 = SVC(random_state = 42)
model3.fit(X_train_3, y_train_3)
predictions3 = model3.predict(X_test_3)
accuracy = accuracy_score(y_test_3, predictions3)
report = classification_report(y_test_3, predictions3)
print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)
print(predictions3)

Accuracy: 0.5
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         6
           1       0.50      1.00      0.67         6

    accuracy                           0.50        12
   macro avg       0.25      0.50      0.33        12
weighted avg       0.25      0.50      0.33        12

[1 1 1 1 1 1 1 1 1 1 1 1]


/Users/eddieloyd/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/eddieloyd/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/eddieloyd/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [408]:
# Train 4th round model
X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(X_4, Y_4, test_size=0.2, random_state=42)
model4 = SVC(random_state = 42)
model4.fit(X_train_4, y_train_4)
predictions4 = model4.predict(X_test_4)
accuracy = accuracy_score(y_test_4, predictions4)
report = classification_report(y_test_4, predictions4)
print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)
print(predictions4)

Accuracy: 0.3333333333333333
Classification Report:
              precision    recall  f1-score   support

           0       0.50      0.25      0.33         4
           1       0.25      0.50      0.33         2

    accuracy                           0.33         6
   macro avg       0.38      0.38      0.33         6
weighted avg       0.42      0.33      0.33         6

[1 1 0 1 0 1]


In [409]:
# Train 5th round model
X_train_5, X_test_5, y_train_5, y_test_5 = train_test_split(X_5, Y_5, test_size=0.01, random_state=42)
model5 = DecisionTreeClassifier(random_state = 42)
model5.fit(X_train_5, y_train_5)
predictions5 = model5.predict(X_test_5)
accuracy = accuracy_score(y_test_5, predictions5)
report = classification_report(y_test_5, predictions5)
print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)
print(predictions5)

Accuracy: 1.0
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1

    accuracy                           1.00         1
   macro avg       1.00      1.00      1.00         1
weighted avg       1.00      1.00      1.00         1

[0]


In [411]:
def get_bracket_setup(which_round):
    """Sets up the bracket data for a certain round, passed in by pd.Dataframe with data
    from the single round
    Randomized so different seeds are listed first and not just higher seed 1st every time
    
    """
    round_results = []
    for ind in which_round.index:
        team1 = which_round["School"][ind]
        score1 = which_round["Score"][ind]
        team2 = which_round["School.1"][ind]
        score2 = which_round["Score.1"][ind]
        
        region_name = which_round["Region Name"][ind]
        path = which_round["Seed"][ind]
        
        r1 = random.randint(0, 100)
        if r1 %2 == 0:
            round_results.append([(team1, team2),  (path, 17-path), region_name])
        else:
            round_results.append([(team2, team1),  (17- path, path), region_name])
    return round_results

In [412]:
# Get the bracket setup
bracket = []
for i in range(1,7):
    round_i = rounds_2017.loc[rounds_2017["Round"] == i]
    bracket.append(get_bracket_setup(round_i))
print(bracket)

[[[('virginia', 'maryland-baltimore-county'), (1.0, 16.0), 'South'], [('cincinnati', 'georgia-state'), (2.0, 15.0), 'South'], [('tennessee', 'wright-state'), (3.0, 14.0), 'South'], [('arizona', 'buffalo'), (4.0, 13.0), 'South'], [('davidson', 'kentucky'), (12.0, 5.0), 'South'], [('loyola-(il)', 'miami-(fl)'), (11.0, 6.0), 'South'], [('texas', 'nevada'), (10.0, 7.0), 'South'], [('creighton', 'kansas-state'), (8.0, 9.0), 'South'], [('texas-southern', 'xavier'), (16.0, 1.0), 'West'], [('lipscomb', 'north-carolina'), (15.0, 2.0), 'West'], [('montana', 'michigan'), (14.0, 3.0), 'West'], [('gonzaga', 'unc-greensboro'), (4.0, 13.0), 'West'], [('ohio-state', 'south-dakota-state'), (5.0, 12.0), 'West'], [('houston', 'san-diego-state'), (6.0, 11.0), 'West'], [('providence', 'texas-a&m'), (10.0, 7.0), 'West'], [('florida-state', 'missouri'), (9.0, 8.0), 'West'], [('radford', 'villanova'), (16.0, 1.0), 'East'], [('purdue', 'cal-state-fullerton'), (2.0, 15.0), 'East'], [('texas-tech', 'stephen-f.-a

In [413]:
from pandas import DataFrame
def simulate_round(single_round, model_name, connecting_data, year):
    """ Simulates a round of the tournament based on the prediction of model inputted
    """
    # Making data able to predict data and move it to the next round, to simulate tournament
    next_round = []
    eliminated_teams = []
    for game in single_round:
        team1, team2 = game[0]
        seed1, seed2 = game[1]
        region = game[2]
                
        team1_data = connecting_data[year][team1]
        team2_data = connecting_data[year][team2]
        team_data = team1_data+ team2_data
        size  = len(team_data)
        team_data = np.array(team_data)
        
        team_data = team_data.reshape(-1, size)
        
        pred = model_name.predict(team_data)
        
        outcome = pred[0].round()
        print(pred)
        # Should predict for which team to win

        
        result = []
        if outcome:
            result.append((team1, (region, seed1)))
            eliminated_teams.append(team2)
        
        else:
            result.append((team2, (region, seed2)))
            eliminated_teams.append(team1)
            
        next_round.append(result)
        
    
    return next_round, eliminated_teams

            

In [378]:
def reduce_first_round(simulated_round):
    """ Reduces first round predictions into a format for the second round
    """
    south =[0] * 4 
    west = [0] * 4 
    east = [0] * 4 
    midwest = [0] * 4 
    for game in simulated_round:
        team , (region, seed) = game[0]
        if seed in (1, 8, 9, 16):
            ind = 0
        elif seed in (2, 7, 10, 15):
            ind = 1
        elif seed in (3, 6, 11, 14):
            ind = 2
        elif seed in (4, 5, 12, 13):
            ind = 3
            
        if region == "South":
            if isinstance(south[ind], list):
                current = south[ind]
                cur_team = current[0]
                current[0] = (cur_team, team)
                cur_seed = current[1]
                current[1] = (cur_seed, seed)
            else:
                south[ind] = [team, seed, region]
                
                
        elif region == "West":
            if isinstance(west[ind], list):
                current = west[ind]
                cur_team = current[0]
                current[0] = (cur_team, team)
                cur_seed = current[1]
                current[1] = (cur_seed, seed)
            else:
                west[ind] = [team, seed, region]
                
        elif region == "East":
            if isinstance(east[ind], list):
                current = east[ind]
                cur_team = current[0]
                current[0] = (cur_team, team)
                cur_seed = current[1]
                current[1] = (cur_seed, seed)
            else:
                east[ind] = [team, seed, region]
        elif region == "Midwest":
            if isinstance(midwest[ind], list):
                current = midwest[ind]
                cur_team = current[0]
                current[0] = (cur_team, team)
                cur_seed = current[1]
                current[1] = (cur_seed, seed)
            else:
                midwest[ind] = [team, seed, region]
    
    final = []
    for i, game in enumerate(south):
        final.append(game)
        final.append(west[i])
        final.append(east[i])
        final.append(midwest[i])
    return final

In [414]:
def reduce_second_round(simulated_round):
    """ Reduces second round predictions into a format for the third round
    """
    south =[0] * 2 
    west = [0] * 2 
    east = [0] * 2 
    midwest = [0] * 2 
    for game in simulated_round:
        team , (region, seed) = game[0]
        if seed in (1, 8, 9, 16, 4, 5, 12, 13):
            ind = 0
        elif seed in (2, 7, 10, 15,3, 6, 11, 14):
            ind = 1
            
        if region == "South":
            if isinstance(south[ind], list):
                current = south[ind]
                cur_team = current[0]
                current[0] = (cur_team, team)
                cur_seed = current[1]
                current[1] = (cur_seed, seed)
            else:
                south[ind] = [team, seed, region]
                
                
        elif region == "West":
            if isinstance(west[ind], list):
                current = west[ind]
                cur_team = current[0]
                current[0] = (cur_team, team)
                cur_seed = current[1]
                current[1] = (cur_seed, seed)
            else:
                west[ind] = [team, seed, region]
                
        elif region == "East":
            if isinstance(east[ind], list):
                current = east[ind]
                cur_team = current[0]
                current[0] = (cur_team, team)
                cur_seed = current[1]
                current[1] = (cur_seed, seed)
            else:
                east[ind] = [team, seed, region]
        elif region == "Midwest":
            if isinstance(midwest[ind], list):
                current = midwest[ind]
                cur_team = current[0]
                current[0] = (cur_team, team)
                cur_seed = current[1]
                current[1] = (cur_seed, seed)
            else:
                midwest[ind] = [team, seed, region]
    
    final = []
    for i, game in enumerate(south):
        final.append(game)
        final.append(west[i])
        final.append(east[i])
        final.append(midwest[i])
    return final

In [415]:
def reduce_third_round(simulated_round):
    """ Reduces third round predictions into a format for the fourth round
    """
    south =[0] 
    west = [0]
    east = [0]
    midwest = [0]
    for game in simulated_round:
        team , (region, seed) = game[0]
        
            
        if region == "South":
            if isinstance(south[0], list):
                current = south[0]
                cur_team = current[0]
                current[0] = (cur_team, team)
                cur_seed = current[1]
                current[1] = (cur_seed, seed)
            else:
                south[0] = [team, seed, region]
                
                
        elif region == "West":
            if isinstance(west[0], list):
                current = west[0]
                cur_team = current[0]
                current[0] = (cur_team, team)
                cur_seed = current[1]
                current[1] = (cur_seed, seed)
            else:
                west[0] = [team, seed, region]
                
        elif region == "East":
            if isinstance(east[0], list):
                current = east[0]
                cur_team = current[0]
                current[0] = (cur_team, team)
                cur_seed = current[1]
                current[1] = (cur_seed, seed)
            else:
                east[0] = [team, seed, region]
        elif region == "Midwest":
            if isinstance(midwest[0], list):
                current = midwest[0]
                cur_team = current[0]
                current[0] = (cur_team, team)
                cur_seed = current[1]
                current[1] = (cur_seed, seed)
            else:
                midwest[0] = [team, seed, region]
    
    final = []
    for i, game in enumerate(south):
        final.append(game)
        final.append(west[i])
        final.append(east[i])
        final.append(midwest[i])
    return final

In [416]:
def reduce_final_four(simulated_round):
    """ Reduces fourth round predictions into a format for the fifth round
    """
    final_four = [0] * 2
    for game in simulated_round:
        team , (region, seed) = game[0]
        
        # South plays east so put them in the same game
        if region == "South":
            if isinstance(final_four[0], list):
                current = final_four[0]
                cur_team = current[0]
                current[0] = (cur_team, team)
                cur_seed = current[1]
                current[1] = (cur_seed, seed)
            else:
                final_four[0] = [team, seed, region]
                
        # West plays midwest so put them in the same game
        elif region == "West":
            if isinstance(final_four[1], list):
                current = final_four[1]
                cur_team = current[0]
                current[0] = (cur_team, team)
                cur_seed = current[1]
                current[1] = (cur_seed, seed)
            else:
                final_four[1] = [team, seed, region]
                
        elif region == "East":
            if isinstance(final_four[0], list):
                current = final_four[0]
                cur_team = current[0]
                current[0] = (cur_team, team)
                cur_seed = current[1]
                current[1] = (cur_seed, seed)
            else:
                final_four[0] = [team, seed, region]
                
        elif region == "Midwest":
            if isinstance(final_four[1], list):
                current = final_four[1]
                cur_team = current[0]
                current[0] = (cur_team, team)
                cur_seed = current[1]
                current[1] = (cur_seed, seed)
            else:
                final_four[1] = [team, seed, region]
    
    return final_four

In [382]:
def reduce_championship(simulated_round):
    """ Reduces final four predictions into a format for the 
    """
    championship = [0]
    for game in simulated_round:
        team , (region, seed) = game[0]
        if isinstance(championship[0], list):
            current = championship[0]
            cur_team = current[0]
            current[0] = (cur_team, team)
            cur_seed = current[1]
            current[1] = (cur_seed, seed)
        else:
            championship[0] = [team, seed, region]
                
    return championship

In [431]:
def simulate_tournament(first_round, model_name_1,model_name_2,model_name_3,model_name_4,model_name_5, connecting_data, year):
    """ Simulates the whole tournament with predictions.
    Each model input corresponds to model used in that round model_1 in 1st round, etc.
    Returns the rounds that teams are eliminated
    
    """

    round_eliminated = []
    round_64_games, eliminated = simulate_round(first_round, model_name_1, connecting_data, year)
    round_64_complete = reduce_first_round(round_64_games)
    round_eliminated.append(eliminated)
    print("Round of 64 winners: ")
    print(round_64_games)
    print("---------------------------------")
    
    round_32_games, eliminated = simulate_round(round_64_complete, model_name_2, connecting_data, year)
    round_eliminated.append(eliminated)
    round_32_complete = reduce_second_round(round_32_games)
    print("Round of 32 winners: ")
    print(round_32_games)
    print("---------------------------------")
    
    round_16_games, eliminated = simulate_round(round_32_complete, model_name_3, connecting_data, year)
    round_eliminated.append(eliminated)
    round_16_complete = reduce_third_round(round_16_games)
    print("Sweet 16 winners: ")
    print(round_16_games)
    print("---------------------------------")
    
    elite_8_games, eliminated = simulate_round(round_16_complete, model_name_4, connecting_data, year)
    round_eliminated.append(eliminated)
    elite_8_complete = reduce_final_four(elite_8_games)
    print("Elite 8 winners: ")
    print(elite_8_games)
    print("---------------------------------")
    
    final_four_complete, eliminated = simulate_round(elite_8_complete, model_name_5, connecting_data, year)
    round_eliminated.append(eliminated)
    print("Final 4 winners: ")
    print(final_four_complete)
    print("---------------------------------")
    
    championship_round = reduce_championship(final_four_complete)
    championship_complete, eliminated = simulate_round(championship_round, model_name_5, connecting_data, year)
    round_eliminated.append(eliminated)
    round_eliminated.append([championship_complete[0][0][0]])
    print("Championship results: ")
    print(championship_complete)
    print("---------------------------------")
    return round_eliminated



In [384]:
# Runs tournament simulation
rounds_predicted = simulate_tournament(bracket[0], model1, model2, model3, model4, model5, connecting_data, "2012-13")

[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
Round of 64 winners: 
[[('virginia', ('South', 1.0))], [('cincinnati', ('South', 2.0))], [('tennessee', ('South', 3.0))], [('arizona', ('South', 4.0))], [('kentucky', ('South', 5.0))], [('miami-(fl)', ('South', 6.0))], [('nevada', ('South', 7.0))], [('creighton', ('South', 8.0))], [('texas-southern', ('West', 16.0))], [('lipscomb', ('West', 15.0))], [('montana', ('West', 14.0))], [('gonzaga', ('West', 4.0))], [('south-dakota-state', ('West', 12.0))], [('san-diego-state', ('West', 11.0))], [('texas-a&m', ('West', 7.0))], [('missouri', ('West', 8.0))], [('radford', ('East', 16.0))], [('cal-state-fullerton', ('East', 15.0))], [('stephen-f.-austin', ('East', 14.0))], [('wichita-state', ('East', 4.0))], [('murray-state', ('East', 12.0))], [('st.-bonaventure', ('East', 11.0))], [('butler', ('East', 10.0))], [('alabama', ('East', 9.0))], [('kansas', ('Midwest', 1.0))

In [417]:
# Returns the round a certain team made it to
round_finished = {}
for i, a_round in enumerate(rounds_predicted):
    for team in a_round:
        if i > 3:
            round_finished[team] = i + 3
        else:
            round_finished[team] = i + 2
print(round_finished)
    

{'maryland-baltimore-county': 2, 'georgia-state': 2, 'wright-state': 2, 'buffalo': 2, 'davidson': 2, 'loyola-(il)': 2, 'texas': 2, 'kansas-state': 2, 'xavier': 2, 'north-carolina': 2, 'michigan': 2, 'unc-greensboro': 2, 'ohio-state': 2, 'houston': 2, 'providence': 2, 'florida-state': 2, 'villanova': 2, 'purdue': 2, 'texas-tech': 2, 'marshall': 2, 'west-virginia': 2, 'florida': 2, 'arkansas': 2, 'virginia-tech': 2, 'pennsylvania': 2, 'duke': 2, 'bucknell': 2, 'college-of-charleston': 2, 'new-mexico-state': 2, 'tcu': 2, 'oklahoma': 2, 'nc-state': 2, 'creighton': 3, 'missouri': 3, 'alabama': 3, 'seton-hall': 3, 'nevada': 3, 'texas-a&m': 3, 'butler': 3, 'rhode-island': 3, 'miami-(fl)': 3, 'san-diego-state': 3, 'st.-bonaventure': 3, 'syracuse': 3, 'kentucky': 3, 'south-dakota-state': 3, 'murray-state': 3, 'clemson': 3, 'arizona': 4, 'gonzaga': 4, 'wichita-state': 4, 'auburn': 4, 'tennessee': 4, 'lipscomb': 4, 'cal-state-fullerton': 4, 'michigan-state': 4, 'virginia': 5, 'texas-southern': 5,

In [432]:
# Returns a bracket with all winners, so it is possible to extract results
def get_bracket_results(which_round):
    round_results = []
    for ind in which_round.index:
        team1 = which_round["School"][ind]
        score1 = which_round["Score"][ind]
        team2 = which_round["School.1"][ind]
        score2 = which_round["Score.1"][ind]
        
        region_name = which_round["Region Name"][ind]
        path = which_round["Seed"][ind]
        

        if score1 > score2:
            winner = (1,0)
        else:
            winner = (0,1)
        round_results.append([(team1, team2), winner, (path, 17-path), region_name])

    
    return round_results

In [419]:
# Extract the results from the bracket 
full_bracket_results = []
for i in range(1,7):
    round_i = rounds_2017.loc[rounds_2017["Round"] == i]
    full_bracket_results.append(get_bracket_results(round_i))
print(full_bracket_results)

[[[('virginia', 'maryland-baltimore-county'), (0, 1), (1.0, 16.0), 'South'], [('cincinnati', 'georgia-state'), (1, 0), (2.0, 15.0), 'South'], [('tennessee', 'wright-state'), (1, 0), (3.0, 14.0), 'South'], [('arizona', 'buffalo'), (0, 1), (4.0, 13.0), 'South'], [('kentucky', 'davidson'), (1, 0), (5.0, 12.0), 'South'], [('miami-(fl)', 'loyola-(il)'), (0, 1), (6.0, 11.0), 'South'], [('nevada', 'texas'), (1, 0), (7.0, 10.0), 'South'], [('creighton', 'kansas-state'), (0, 1), (8.0, 9.0), 'South'], [('xavier', 'texas-southern'), (1, 0), (1.0, 16.0), 'West'], [('north-carolina', 'lipscomb'), (1, 0), (2.0, 15.0), 'West'], [('michigan', 'montana'), (1, 0), (3.0, 14.0), 'West'], [('gonzaga', 'unc-greensboro'), (1, 0), (4.0, 13.0), 'West'], [('ohio-state', 'south-dakota-state'), (1, 0), (5.0, 12.0), 'West'], [('houston', 'san-diego-state'), (1, 0), (6.0, 11.0), 'West'], [('texas-a&m', 'providence'), (1, 0), (7.0, 10.0), 'West'], [('missouri', 'florida-state'), (0, 1), (8.0, 9.0), 'West'], [('villa

In [420]:
# Obtains the actual rounds eliminated
rounds_eliminated = []
for full_round in full_bracket_results:
    round_eliminated = []
    for game in full_round:
        if game[1][0] == 0:
            round_eliminated.append(game[0][0])
            if full_round == full_bracket_results[5]:
                rounds_eliminated.append(round_eliminated)
                rounds_eliminated.append([game[0][1]])
        elif game[1][1] == 0:
            round_eliminated.append(game[0][1])
            if full_round == full_bracket_results[5]:
                rounds_eliminated.append(round_eliminated)
                rounds_eliminated.append([game[0][0]])
    if full_round != full_bracket_results[5]:    
        rounds_eliminated.append(round_eliminated)


    
print(rounds_eliminated)


[['virginia', 'georgia-state', 'wright-state', 'arizona', 'davidson', 'miami-(fl)', 'texas', 'creighton', 'texas-southern', 'lipscomb', 'montana', 'unc-greensboro', 'south-dakota-state', 'san-diego-state', 'providence', 'missouri', 'radford', 'cal-state-fullerton', 'stephen-f.-austin', 'wichita-state', 'murray-state', 'st.-bonaventure', 'arkansas', 'virginia-tech', 'pennsylvania', 'iona', 'bucknell', 'college-of-charleston', 'new-mexico-state', 'tcu', 'oklahoma', 'nc-state'], ['maryland-baltimore-county', 'cincinnati', 'tennessee', 'buffalo', 'xavier', 'north-carolina', 'houston', 'ohio-state', 'alabama', 'butler', 'florida', 'marshall', 'seton-hall', 'rhode-island', 'michigan-state', 'auburn'], ['kentucky', 'nevada', 'gonzaga', 'texas-a&m', 'west-virginia', 'purdue', 'clemson', 'syracuse'], ['kansas-state', 'florida-state', 'texas-tech', 'duke'], ['loyola-(il)', 'kansas'], ['michigan'], ['villanova']]


In [421]:
# Obtains the actual rounds eliminated in dict format
round_finished_actual = {}
for i, a_round in enumerate(rounds_eliminated):
    for team in a_round:
        if i > 3:
            round_finished_actual[team] = i + 3
        else:
            round_finished_actual[team] = i + 2


In [422]:
# Creates predictions vs actual results
team_names = list(set(round_finished_actual.keys()))
team_names.sort()
predictions = []
actual = []
for team in team_names:
    predictions.append(round_finished[team])
    actual.append(round_finished_actual[team])
    

In [433]:
# Compares the predictions and actual results
accuracy = accuracy_score(actual, predictions)
report = classification_report(actual, predictions)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)


Accuracy: 0.3125
Classification Report:
              precision    recall  f1-score   support

           2       0.47      0.47      0.47        32
           3       0.25      0.25      0.25        16
           4       0.12      0.12      0.12         8
           5       0.00      0.00      0.00         4
           7       0.00      0.00      0.00         2
           8       0.00      0.00      0.00         1
           9       0.00      0.00      0.00         1

    accuracy                           0.31        64
   macro avg       0.12      0.12      0.12        64
weighted avg       0.31      0.31      0.31        64

